In [16]:
!apt-get install -y coinor-cbc
!pip install pyomo


'apt-get' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
import pyomo.environ as pyo
from itertools import product

# Cargar archivos
vehicles = pd.read_csv("Datos\Vehicles.csv")
stations = pd.read_csv("Datos\stations.csv")  # No se usa en Caso 1
depots = pd.read_csv("Datos\depots.csv")
clients = pd.read_csv("Datos\clients.csv")

# Conjuntos
V = list(vehicles["VehicleID"].astype(str))
E = list(stations["EstationID"].astype(str))
P = list(depots["DepotID"].astype(str))
D = list(clients["LocationID"].astype(str))
N = P + D + E
A = [(i, j) for i in N for j in N if i != j]

# Parámetros
M_v = dict(zip(V, vehicles["Capacity"]))
A_v = dict(zip(V, vehicles["Range"]))
R_v = {v: 4 for v in V}
L_j = {j: 99999 for j in D}  # Asignar límites por defecto
demand = dict(zip(clients["LocationID"].astype(str), clients["Demand"]))
P_s = dict(zip(stations["EstationID"].astype(str), stations["FuelCost"]))

# Costos
dist = {(i, j): 1 for (i, j) in A}
C_ij = {(i, j): 1000 for (i, j) in A}
T_ij = {(i, j): 2000 for (i, j) in A}
T2_ij = {(i, j): 50 for (i, j) in A}
F_t = 5000
C_m = 700
E_v = {v: 10 for v in V}


<>:6: SyntaxWarning: invalid escape sequence '\V'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\V'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\c'
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_31996\2839009681.py:6: SyntaxWarning: invalid escape sequence '\V'
  vehicles = pd.read_csv("Datos\Vehicles.csv")
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_31996\2839009681.py:7: SyntaxWarning: invalid escape sequence '\s'
  stations = pd.read_csv("Datos\stations.csv")  # No se usa en Caso 1
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_31996\2839009681.py:8: SyntaxWarning: invalid escape sequence '\d'
  depots = pd.read_csv("Datos\depots.csv")
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_31996\2839009681.py:9: SyntaxWa

In [18]:
model = pyo.ConcreteModel()
model.N = pyo.Set(initialize=N)
model.V = pyo.Set(initialize=V)
model.A = pyo.Set(dimen=2, initialize=A)
model.E = pyo.Set(initialize=E)
model.D = pyo.Set(initialize=D)

model.x = pyo.Var(model.A, model.V, domain=pyo.Binary)
model.q = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)
model.z = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.E, model.V, domain=pyo.Binary)
model.r = pyo.Var(model.E, model.V, domain=pyo.NonNegativeReals)
model.f = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)
model.comb = pyo.Var(model.N, model.V, domain=pyo.NonNegativeReals)
model.u = pyo.Var(model.D, model.V, domain=pyo.NonNegativeReals)

# Inicialización de combustible
for v in V:
    for p in P:
        model.comb[p, v].fix(A_v[v])
    for n in N:
        if n not in P:
            model.comb[n, v].setlb(0)

model.z_restr = pyo.ConstraintList()
for (i, j) in A:
    for v in V:
        model.z_restr.add(model.z[i, j, v] <= model.q[i, j, v])
        model.z_restr.add(model.z[i, j, v] <= M_v[v] * model.x[i, j, v])
        model.z_restr.add(model.z[i, j, v] >= model.q[i, j, v] - M_v[v] * (1 - model.x[i, j, v]))


In [19]:
model.obj = pyo.Objective(
    expr=sum(
        (C_ij[i, j] + F_t * dist[i, j] + C_m * dist[i, j] + T_ij[i, j] + E_v[v] * dist[i, j]) * model.x[i, j, v] +
        T2_ij[i, j] * model.z[i, j, v]
        for (i, j) in model.A for v in model.V
    ) + sum(P_s[s] * model.r[s, v] for s in model.E for v in model.V),
    sense=pyo.minimize
)

def flujo_conservacion(model, n, v):
    return sum(model.x[i, n, v] for i in N if i != n) == sum(model.x[n, j, v] for j in N if j != n)
model.flujo = pyo.Constraint(N, model.V, rule=flujo_conservacion)

def restr_capacidad(model, i, j, v):
    return model.q[i, j, v] <= M_v[v] * model.x[i, j, v]
model.capacidad = pyo.Constraint(model.A, model.V, rule=restr_capacidad)

def restr_peso(model, i, j, v):
    if j in L_j:
        return model.q[i, j, v] <= L_j[j]
    return pyo.Constraint.Skip
model.peso = pyo.Constraint(model.A, model.V, rule=restr_peso)

def consumo_combustible(model, i, j, v):
    return model.f[i, j, v] == (dist[i, j] / R_v[v]) * model.x[i, j, v]
model.consumo = pyo.Constraint(model.A, model.V, rule=consumo_combustible)

def continuidad_combustible(model, i, j, v):
    return model.comb[j, v] == model.comb[i, v] - model.f[i, j, v] + (model.r[j, v] if j in E else 0)
model.continuidad = pyo.Constraint(model.A, model.V, rule=continuidad_combustible)

def recarga_estacion(model, s, v):
    return model.r[s, v] <= 9999 * model.y[s, v]
model.recarga = pyo.Constraint(model.E, model.V, rule=recarga_estacion)

def subtour_eliminacion(model, i, j, v):
    if i != j and i in D and j in D:
        return model.u[i, v] - model.u[j, v] + len(D) * model.x[i, j, v] <= len(D) - 1
    return pyo.Constraint.Skip
model.subtours = pyo.Constraint(model.A, model.V, rule=subtour_eliminacion)

def satisfacer_demanda(model, j):
    return sum(model.q[i, j, v] for i in N if i != j for v in V) >= demand[j]
model.demanda = pyo.Constraint(model.D, rule=satisfacer_demanda)


In [20]:
solver = pyo.SolverFactory('glpk', executable='\glpk-4.65\\w64\\glpsol.exe')
results = solver.solve(model, tee=True)

if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("¡Solución óptima encontrada!")
    print("Valor objetivo:", pyo.value(model.obj))
else:
    print("No se encontró solución óptima.")
    print("Estado:", results.solver.status)
    print("Condición:", results.solver.termination_condition)


<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\MarianaLozano\AppData\Local\Temp\ipykernel_31996\1272832471.py:1: SyntaxWarning: invalid escape sequence '\g'
  solver = pyo.SolverFactory('glpk', executable='\glpk-4.65\\w64\\glpsol.exe')


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\MARIAN~1\AppData\Local\Temp\tmprjyrko85.glpk.raw --wglp
 C:\Users\MARIAN~1\AppData\Local\Temp\tmpzq05bab3.glpk.glp --cpxlp C:\Users\MARIAN~1\AppData\Local\Temp\tmp60zyu4jq.pyomo.lp
Reading problem data from 'C:\Users\MARIAN~1\AppData\Local\Temp\tmp60zyu4jq.pyomo.lp'...
C:\Users\MARIAN~1\AppData\Local\Temp\tmp60zyu4jq.pyomo.lp:91397: warning: lower bound of variable 'x2' redefined
C:\Users\MARIAN~1\AppData\Local\Temp\tmp60zyu4jq.pyomo.lp:91397: warning: upper bound of variable 'x2' redefined
15359 rows, 4460 columns, 38690 non-zeros
1110 integer variables, all of which are binary
92507 lines were read
Writing problem data to 'C:\Users\MARIAN~1\AppData\Local\Temp\tmpzq05bab3.glpk.glp'...
77346 lines were written
GLPK Integer Optimizer, v4.65
15359 rows, 4460 columns, 38690 non-zeros
1110 integer variables, all of which are binary
Preprocessing...
60 constraint coefficient(s) were reduced
14379